## Loading Parking Violation Data and cleaning operations

In [ ]:
import glob
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
files = glob.glob('../vz-hackathon/**')

In [ ]:
parking_violations = [f for f in files if 'Parking_Violations_in' in f]

### Combine all csvs into one data frame

In [ ]:
list_ = []
for file_ in parking_violations[:]:
    df = pd.read_csv(file_,index_col=None, header=0)
    
    filename = file_[len('../vz-hackathon/'):]
    
    df['filename'] = filename
       
    list_.append(df)
    
frame = pd.concat(list_)

# TESTs
assert frame.filename.nunique() == len(parking_violations)
frame.columns = [col.lower() for col in frame.columns]
frame = frame.reset_index(drop=True)
df = frame.copy()

### Construct datetime object from issue_time raw string

In [ ]:
def mil_to_time(x):
    "Convert messy issue_time to datetime object based upon length of issue_time string"
    if x == 'nan':
        return '00:00:00.000Z'
        
    x = x.split('.')[0]
    lg = len(x)
    
    if lg == 4:
        t = x[:2] + ':' + x[2:] + ':00.000Z'
      
    elif lg == 3:
        t = '0' + x[0] + ':' + x[1:] + ':00.000Z'
    
    elif lg == 2:
        t = '0' + '0' + ':' + x + ':00.000Z'
    
    elif lg == 1:
        t = '0' + '0' + ':' + '0' + x + ':00.000Z'
    
    else:
        t = '00:00.000Z'
    
    # correction for timedate if one element is greater than 5.
    # double check this
    if int(t[3]) > 5:
        t = t[:2]+ ':' + '5' + t[4:]
        
    return t

In [ ]:
df['issue_time_military'] = df.issue_time.apply(str).apply(mil_to_time)
dates = df.ticket_issue_date.str[:10] + 'T' #+ 
df['ticket_issue_datetime'] = dates + df.issue_time_military

### Optional Test that datetime contruction is correct

In [ ]:
## Testing datetime format to ensure that everything is actually datetime
# for i, t in enumerate(df.ticket_issue_datetime):
#     try:
#         pd.to_datetime(t)
#     except:
#         print i, t

### Holiday value to Boolean

In [ ]:
df['holiday'] = df.holiday != 0

### Delete redundant columns

In [ ]:
del df['day_of_week']
del df['month_of_year']
del df['week_of_year']
del df['issue_time']
del df['issue_time_military']
del df['ticket_issue_date']

### Drop Duplicates and Fill in Empy Rows

In [ ]:
df.drop_duplicates(subset='rowid_', inplace=True)
df.streetsegid.fillna(0, inplace=True)

### Export to CSV

In [ ]:
df.to_csv('../cleaned_data/clean_parking_violations.csv', sep='\t', index=False)

In [ ]:
df.sample(10000).to_csv('../sampled_data/clean_parking_violations_10k_sample.csv', sep='\t', index=False)

In [ ]:
df.sample(100000).to_csv('../sampled_data/clean_parking_violations_100k_sample.csv', sep='\t', index=False)

In [ ]:
df.sample(1000000).to_csv('../sampled_data/clean_parking_violations_1million_sample.csv', sep='\t', index=False)